In [4]:
import sys
sys.path.append('..')
import generate_potential_2d_spiral as gp2Sp
import generate_potential_2d_Zshape as gp2Zs
import generate_trajectory_Langevin_2D_from_symbolic as gt2La

In [5]:
dir(gp2Sp)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'cm',
 'generate_potential_2d_spirals_numeric',
 'generate_potential_2d_spirals_symbolic',
 'np',
 'plt',
 'spy',
 'stats',
 'tqdm']

In [6]:
potential_Spiral_symbolic, _, _ = gp2Sp.generate_potential_2d_spirals_symbolic([100,100],2,False,True)

TypeError: generate_potential_2d_spirals_symbolic() takes 3 positional arguments but 4 were given